## 1. Data loading

In [29]:
import pandas as pd
df = pd.read_csv("extracted_processed.csv")
df

,title with text,label,prcessed_text,sentiment,word_count,sentence_count,avg_sentence_length
0,"As U.S. budget fight looms, Republicans flip t...",1,as u s budget fight loom republicans flip thei...,True,759,52,14.596154
1,U.S. military to accept transgender recruits o...,1,u s military to accept transgender recruit on ...,False,633,40,15.825000
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1,senior u s republican senator let mr mueller d...,False,467,26,17.961538
3,FBI Russia probe helped by Australian diplomat...,1,fbi russia probe help by australian diplomat t...,True,385,22,17.500000
4,Trump wants Postal Service to charge 'much mor...,1,trump want postal service to charge much more ...,True,863,59,14.627119
...,...,...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,0,mcpain john mccain furious that iran treat us ...,True,532,19,28.000000
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0,justice yahoo settle e mail privacy class acti...,True,309,13,23.769231
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,0,sunnistan us and ally safe zone plan to take t...,False,4185,141,29.680851
44896,How to Blow $700 Million: Al Jazeera America F...,0,how to blow million al jazeera america finally...,True,468,22,21.272727


## 2. Data cleaning

In [2]:
df = df.dropna()
df

,title with text,label,prcessed_text,sentiment,word_count,sentence_count,avg_sentence_length
0,"As U.S. budget fight looms, Republicans flip t...",1,as u s budget fight loom republicans flip thei...,True,759,52,14.596154
1,U.S. military to accept transgender recruits o...,1,u s military to accept transgender recruit on ...,False,633,40,15.825000
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1,senior u s republican senator let mr mueller d...,False,467,26,17.961538
3,FBI Russia probe helped by Australian diplomat...,1,fbi russia probe help by australian diplomat t...,True,385,22,17.500000
4,Trump wants Postal Service to charge 'much mor...,1,trump want postal service to charge much more ...,True,863,59,14.627119
...,...,...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,0,mcpain john mccain furious that iran treat us ...,True,532,19,28.000000
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0,justice yahoo settle e mail privacy class acti...,True,309,13,23.769231
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,0,sunnistan us and ally safe zone plan to take t...,False,4185,141,29.680851
44896,How to Blow $700 Million: Al Jazeera America F...,0,how to blow million al jazeera america finally...,True,468,22,21.272727


Select the locations that have a complete set of data (12 dates, from 02/01 to 07/15).

,title with text,label,prcessed_text,sentiment,word_count,sentence_count,avg_sentence_length
0,"As U.S. budget fight looms, Republicans flip t...",1,as u s budget fight loom republicans flip thei...,True,759,52,14.596154
1,U.S. military to accept transgender recruits o...,1,u s military to accept transgender recruit on ...,False,633,40,15.825000
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1,senior u s republican senator let mr mueller d...,False,467,26,17.961538
3,FBI Russia probe helped by Australian diplomat...,1,fbi russia probe help by australian diplomat t...,True,385,22,17.500000
4,Trump wants Postal Service to charge 'much mor...,1,trump want postal service to charge much more ...,True,863,59,14.627119
...,...,...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,0,mcpain john mccain furious that iran treat us ...,True,532,19,28.000000
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0,justice yahoo settle e mail privacy class acti...,True,309,13,23.769231
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,0,sunnistan us and ally safe zone plan to take t...,False,4185,141,29.680851
44896,How to Blow $700 Million: Al Jazeera America F...,0,how to blow million al jazeera america finally...,True,468,22,21.272727


## 3. Classification

[Test:] Can we predict whether "new_cases" on 07/15 is bigger than "new_cases" on 07/01 based on historical data (8 dates: 03/15, 04/01, 04/15, 05/01, 05/15, 06/01, 06/15, 07/01)?

[Train:] "new_cases" **bigger?** as label on 07/01 (than 06/15), 06/15 (than 06/01), 06/01 (than 05/15).
* If label date = 07/01, Features: 8 dates: 03/01, 03/15, 04/01, 04/15, 05/01, 05/15, 06/01, 06/15.
* If label date = 06/15, Features: 8 dates: 02/15, 03/01, 03/15, 04/01, 04/15, 05/01, 05/15, 06/01.
* If label date = 06/01, Features: 8 dates: 02/01, 02/15, 03/01, 03/15, 04/01, 04/15, 05/01, 05/15.

In [81]:
feature_names = [#"title with text", 
                 #'prcessed_text', 
                 'sentiment', 
                 'word_count',
                 'sentence_count',
                 'avg_sentence_length']
import numpy as np


label_1 = df[df['label'] == 1] # 21417 rows
label_0 = df[df['label'] == 0] # 23481 rows

total_rows = df.shape[0]
training_rows = int(df.shape[0] * 0.8) #35918
testing_rows = int(df.shape[0] - training_rows)

training_1_int = int(0.8 * label_1.shape[0])
training_0_int = int(0.8 * label_0.shape[0])

train_1 = label_1.iloc[:training_1_int]
train_0 = label_0.iloc[:training_0_int]
test_1 = label_1.iloc[training_1_int:]
test_0 = label_1.iloc[training_0_int:]

training = pd.concat([train_1, train_0], sort=False)
training = training.sample(frac=1).reset_index(drop=True)
test = pd.concat([test_1, test_0], sort=False)
test = test.sample(frac=1).reset_index(drop=True)

X_train = training[feature_names]
y_train = training['label']

X_test = test[feature_names]
y_test = test['label']


In [82]:
import numpy as np


X_train, X_test = np.array(X_train), np.array(X_test)

print("X_train.shape", X_train.shape)
print("y_train.shape", len(y_train), "#positives (more cases):", sum(y_train))
print("X_test.shape", X_test.shape)
print("y_test.shape", len(y_test), "#positives (more cases):", sum(y_test))

X_train.shape (35917, 4)
y_train.shape 35917 #positives (more cases): 17133
X_test.shape (6917, 4)
y_test.shape 6917 #positives (more cases): 6917


In [83]:
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

clf_knn = KNeighborsClassifier()
clf_id3 = tree.DecisionTreeClassifier(criterion="entropy", max_depth=2)
clf_id3_overfit = tree.DecisionTreeClassifier(criterion="entropy")
clf_cart = tree.DecisionTreeClassifier(max_depth=2)
clf_bayes = GaussianNB()
clf_rbf = SVC()
clf_forest = RandomForestClassifier()
clf_boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=2), random_state=21)
clf_mlp = MLPClassifier(hidden_layer_sizes=(50,50,), max_iter=1000, tol=0.001, random_state=42)

clf_knn.fit(X_train, y_train)
clf_id3.fit(X_train, y_train)
clf_id3_overfit.fit(X_train, y_train)
clf_cart.fit(X_train, y_train)
clf_bayes.fit(X_train, y_train)
clf_rbf.fit(X_train, y_train)
clf_forest.fit(X_train, y_train)
clf_boost.fit(X_train, y_train)
clf_mlp.fit(X_train, y_train)

print("Accuracy:")
print("- KNN:", metrics.accuracy_score(y_test, clf_knn.predict(X_test)))
print("- ID3:", metrics.accuracy_score(y_test, clf_id3.predict(X_test)))
print("- ID3 (overfitting):", metrics.accuracy_score(y_test, clf_id3_overfit.predict(X_test)))
print("- CART:", metrics.accuracy_score(y_test, clf_cart.predict(X_test)))
print("- Naive Bayes:", metrics.accuracy_score(y_test, clf_bayes.predict(X_test)))
print("- RBF Kernel SVC:", metrics.accuracy_score(y_test, clf_rbf.predict(X_test)))
print("- Random Forest:", metrics.accuracy_score(y_test, clf_forest.predict(X_test)))
print("- AdaBoost:", metrics.accuracy_score(y_test, clf_boost.predict(X_test)))
print("- MLP:", metrics.accuracy_score(y_test, clf_mlp.predict(X_test)))

Accuracy:
- KNN: 0.510336851236085
- ID3: 0.2746855573225387
- ID3 (overfitting): 0.48720543588260806
- CART: 0.4191123319358103
- Naive Bayes: 0.4992048575972242
- RBF Kernel SVC: 0.5260951279456412
- Random Forest: 0.5395402631198496
- AdaBoost: 0.4565563105392511
- MLP: 0.5661413907763482


## 4. Regression

[Test:] Can we predict "new_cases" on 07/15 based on historical data (8 dates: 03/15, 04/01, 04/15, 05/01, 05/15, 06/01, 06/15, 07/01)?

[Train:] "new_cases" as label on 07/01, 06/15, 06/01.
* If label date = 07/01, Features: 8 dates: 03/01, 03/15, 04/01, 04/15, 05/01, 05/15, 06/01, 06/15.
* If label date = 06/15, Features: 8 dates: 02/15, 03/01, 03/15, 04/01, 04/15, 05/01, 05/15, 06/01.
* If label date = 06/01, Features: 8 dates: 02/01, 02/15, 03/01, 03/15, 04/01, 04/15, 05/01, 05/15.

In [7]:
y_train, y_test = [], []

for location in sorted(set(df["location"])):
    temp = df[df["location"] == location]
    
    newnum = temp[temp["date"] == "2020-07-15"]["new_cases"].to_numpy()[0]
    y_test.append(newnum+1) # --> for log_error
    
    newnum = temp[temp["date"] == "2020-07-01"]["new_cases"].to_numpy()[0]
    y_train.append(newnum+1)
    
    newnum = temp[temp["date"] == "2020-06-15"]["new_cases"].to_numpy()[0]
    y_train.append(newnum+1)

    newnum = temp[temp["date"] == "2020-06-01"]["new_cases"].to_numpy()[0]
    y_train.append(newnum+1)

print("y_train.shape", len(y_train), "min:", min(y_train), "max:", max(y_test))
print("y_test.shape", len(y_test), "min:", min(y_test), "max:", max(y_test))

y_train.shape 186 min: 1.0 max: 68519.0
y_test.shape 62 min: 1.0 max: 68519.0


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

reg_lr = LinearRegression()
reg_knn = KNeighborsRegressor()
reg_cart = tree.DecisionTreeRegressor()
reg_bayes = BayesianRidge()
reg_rbf = SVR()
reg_forest = RandomForestRegressor(random_state=42)
reg_boost = AdaBoostRegressor(random_state=42)
reg_mlp = MLPRegressor(random_state=42)

reg_lr.fit(X_train, y_train)
reg_knn.fit(X_train, y_train)
reg_cart.fit(X_train, y_train)
reg_bayes.fit(X_train, y_train)
reg_rbf.fit(X_train, y_train)
reg_forest.fit(X_train, y_train)
reg_boost.fit(X_train, y_train)
reg_mlp.fit(X_train, y_train)

print("MSLE:")
print("- Linear Regression:", metrics.mean_squared_log_error(y_test, np.absolute(reg_lr.predict(X_test))))
print("- KNN:", metrics.mean_squared_log_error(y_test, np.absolute(reg_knn.predict(X_test))))
print("- CART:", metrics.mean_squared_log_error(y_test, np.absolute(reg_cart.predict(X_test))))
print("- Bayesian Ridge:", metrics.mean_squared_log_error(y_test, np.absolute(reg_bayes.predict(X_test))))
print("- RBF Kernel SVR:", metrics.mean_squared_log_error(y_test, np.absolute(reg_rbf.predict(X_test))))
print("- Random Forest:", metrics.mean_squared_log_error(y_test, np.absolute(reg_forest.predict(X_test))))
print("- AdaBoost:", metrics.mean_squared_log_error(y_test, np.absolute(reg_boost.predict(X_test))))
print("- MLP:", metrics.mean_squared_log_error(y_test, np.absolute(reg_mlp.predict(X_test))))

MSLE:
- Linear Regression: 17.86119446978422
- KNN: 4.0882677615894245
- CART: 1.9722927424175254
- Bayesian Ridge: 6.2328763007592975
- RBF Kernel SVR: 7.184319019445448
- Random Forest: 1.7714774728876155
- AdaBoost: 8.00575351887432
- MLP: 20.123937393917917
